In [3]:
!pip install pandas numpy faker

In [4]:
import pandas as pd
import numpy as np
from faker import Faker
import random

## Generat Customer

In [5]:
fake = Faker()

NUM_CUSTOMERS = 500
NUM_INVOICES = 10000

customers = []

for i in range(NUM_CUSTOMERS):
    customers.append({
        "customer_id": i + 1,
        "company_name": fake.company(),
        "industry": random.choice(
            ["SaaS", "Retail", "Finance", "Healthcare", "Ecommerce"]
        ),
        "country": random.choice(["India", "Singapore", "UAE", "USA"])
    })

customers_df = pd.DataFrame(customers)

## Generate Invoices

In [6]:
NUM_INVOICES = 10000

invoices = []

for i in range(NUM_INVOICES):

    issue_date = fake.date_between(start_date="-1y", end_date="today")
    due_date = pd.to_datetime(issue_date) + pd.Timedelta(days=30)

    invoices.append({
        "invoice_id": i+1,
        "customer_id": random.randint(1, NUM_CUSTOMERS),
        "amount": round(random.uniform(500,50000),2),
        "issue_date": issue_date,
        "due_date": due_date,
        "status": random.choice(["paid","pending","overdue"])
    })

invoices_df = pd.DataFrame(invoices)
invoices_df.head()

,invoice_id,customer_id,amount,issue_date,due_date,status
0,1,52,46247.88,2025-09-11,2025-10-11,overdue
1,2,471,10540.22,2025-06-11,2025-07-11,overdue
2,3,263,6679.74,2025-07-18,2025-08-17,paid
3,4,47,3492.57,2025-06-21,2025-07-21,overdue
4,5,31,11097.84,2026-02-04,2026-03-06,pending


## Generate Payments

In [9]:
invoices = []
payments = []

today = pd.Timestamp.today()

for i in range(NUM_INVOICES):

    issue_date = fake.date_between(start_date="-1y", end_date="today")
    due_date = pd.to_datetime(issue_date) + pd.Timedelta(days=30)

    # decide if invoice gets paid (70% chance)
    is_paid = random.random() < 0.7

    if is_paid:

        # random payment behavior
        payment_type = random.choice(["early", "on_time", "late"])

        if payment_type == "early":
            payment_date = pd.to_datetime(due_date) - pd.Timedelta(
                days=random.randint(1, 10)
            )

        elif payment_type == "on_time":
            payment_date = pd.to_datetime(due_date) + pd.Timedelta(
                days=random.randint(-2, 2)
            )

        else:  # late
            payment_date = pd.to_datetime(due_date) + pd.Timedelta(
                days=random.randint(3, 45)
            )

        # prevent future payment dates
        payment_date = min(payment_date, today)

        status = "paid"

        payments.append({
            "payment_id": len(payments) + 1,
            "invoice_id": i + 1,
            "payment_date": payment_date,
            "amount_paid": round(random.uniform(500, 50000), 2)
        })

    else:
        # unpaid logic
        if due_date > today:
            status = "pending"
        else:
            status = "overdue"

    invoices.append({
        "invoice_id": i + 1,
        "customer_id": random.randint(1, NUM_CUSTOMERS),
        "amount": round(random.uniform(500, 50000), 2),
        "issue_date": issue_date,
        "due_date": due_date,
        "status": status
    })


invoices_df = pd.DataFrame(invoices)
payments_df = pd.DataFrame(payments)

## Save CSV

In [10]:
customers_df.to_csv("../data/raw/customers.csv", index=False)
invoices_df.to_csv("../data/raw/invoices.csv", index=False)
payments_df.to_csv("../data/raw/payments.csv", index=False)

print("All files saved")

All files saved
